# Операции над полями Галуа GF(2^8) в AES.

In [526]:
primitive = 0x1C3  # неприводимый многочлен x^8 + x^7 + x^6 + x + 1
check_high_bit = 0x80 # 10000000

In [527]:
def multiply_GF256(a, b):
    """
    Nhân hai phần tử trong trường GF(2^8) sử dụng đa thức bất khả quy 0x11B.
    """
    result = 0
    for i in range(8):
        if b & 1:  # Nếu bit cuối cùng của b là 1
            result ^= a  # Cộng (XOR) a vào kết quả
        high_bit_set = a & check_high_bit  # Kiểm tra bit cao nhất của a
        a <<= 1  # Dịch trái a
        if high_bit_set:
            a ^= primitive  # Chia lấy dư với đa thức bất khả quy nếu cần
        b >>= 1  # Dịch phải b
    return result

In [528]:
multiply_GF256(2, 148)

235

In [529]:
def inverse_GF256(x):
    """Tìm nghịch đảo của x trong GF(2^8) bằng phương pháp lũy thừa nhanh."""
    if x == 0:
        return 0
    y = 1
    for _ in range(254):  # x^(254) mod GF(2^8) sẽ cho ra nghịch đảo
        y = multiply_GF256(y, x)
    return y

In [530]:
def exp_GF256(elem, exp):
    if exp == 0:
        return 1
    r = exp % 2
    q = exp // 2
    if r:
        return multiply_GF256(elem, multiply_GF256(exp_GF256(elem, q), exp_GF256(elem, q)))
    else:
        return multiply_GF256(exp_GF256(elem, q), exp_GF256(elem, q))

# Подготовка данных

In [531]:
from bitarray import bitarray
from bitarray.util import int2ba, ba2int
import json

In [532]:
def hex_to_bin(hex_string, version=32):
    hex_string = hex_string.replace(" ", "")
    return bitarray(bin(int(hex_string, 16))[2:].zfill(version))

In [533]:
def bin_to_hex(bin_array, version=32):
    """Chuyển đổi dữ liệu nhị phân thành chuỗi thập lục phân với định dạng mong muốn."""
    hex_string = format(ba2int(bin_array), 'X').zfill(version // 4)
    return ' '.join(hex_string[i:i+2] for i in range(0, len(hex_string), 2))

In [534]:
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

In [535]:
S_BOX = read_json('sbox_kuznyechik.json')
L_MATRIX = read_json('L_kuznyechik.json')

# SP-сеть с набором преобразований L, S, X

In [536]:
def apply_S_box(state: bitarray) -> bitarray:
    # chuyển đổi bitarray 16 byte thành list các byte
    state = [state[i:i+8] for i in range(0, len(state), 8)]
    new_state = []
    for byte in state:
        row = ba2int(byte[:4])
        col = ba2int(byte[4:8])
        new_state.append(S_BOX[row][col])
    return bitarray(''.join(format(byte, '08b') for byte in new_state))

In [537]:
tmp = '93'
print(hex_to_bin(tmp, version=8))
bin_to_hex(apply_S_box(hex_to_bin(tmp, version=8)))

bitarray('10010011')


'00 00 00 DE'

In [538]:
def apply_L(state: bitarray) -> bitarray:
    # chuyển đổi bitarray 16 byte thành list các byte
    state = [state[i:i+8] for i in range(0, len(state), 8)]
    state = [ba2int(byte) for byte in state]
    for _ in range(16):
        new_byte = 0
        for j in range(16):
            new_byte ^= multiply_GF256(state[j], L_MATRIX[j])
        state = [new_byte] + state[:15]  # Dịch vòng
    
    # chuyeern state từ list byte thành bitarray
    state = ''.join(format(byte, '08b') for byte in state)
    return bitarray(state)

In [539]:
bin_to_hex(apply_L(hex_to_bin('75 46 70 72 F1 81 81 C7 ED 33 05 B0 DA 3A EC 6C', version=128)))

'C8 C6 CC DD 03 4D 8E 6B D7 33 8E 20 7F 6B 09 71'

In [540]:
def apply_X(state: bitarray, key: bitarray) -> bitarray:
    return state ^ key

# Алгоритм расширения ключа Кузнечик

In [541]:
key = "FD E8 F7 A9 B8 6C 3B FF 07 C0 D3 9D 04 60 5E DD 14 A3 D4 B6 33 45 4D 7C 5B 21 3A 5B 9A 0F 58 6C"

In [542]:
def generate_C(i) -> bitarray:
    C_i = bitarray('0' * 120) + bitarray(int2ba(i, length=8))
    return apply_L(C_i)

In [543]:
print(bin_to_hex(generate_C(7)))

C9 E8 81 9D C7 3B A5 AE 50 F5 B5 70 56 1A 6A 07


In [544]:
def f_function(left: bitarray, C_i: bitarray) -> bitarray:
    print("C_i:\n", bin_to_hex(C_i))
    result = apply_X(left, C_i)
    print("After xor C_i:\n", bin_to_hex(result))
    result = apply_S_box(result)
    print("After apply S_Box:\n", bin_to_hex(result))
    result = apply_L(left)
    print("After apply L:\n", bin_to_hex(result))
    #return apply_X(apply_S_box(apply_L(left)), C_i)
    return result

## Раунды ключи

In [545]:
def RoundKeys(left: bitarray, right: bitarray, C_i: bitarray) -> (bitarray, bitarray):
    next_left = right ^ f_function(left, C_i)
    next_right = left
    return next_left, next_right

In [546]:
key_bin = hex_to_bin('AD 1E EA 2D E2 F0 CE C8 35 C9 24 44 12 B0 B9 58' + 
                     'F6 87 AA 3C 2F 34 A9 F0 2F 1A 9D 5F BB E7 4C 6B', version=256)
left, right = key_bin[:128], key_bin[128:]
print("Start left key:\n", bin_to_hex(left))
print("Start Right key:\n", bin_to_hex(right))
left_, right_ = RoundKeys(left, right, generate_C(7))
print("Left key inter 7:\n", bin_to_hex(left_))
print("Right key inter 7:\n", bin_to_hex(right_))

Start left key:
 AD 1E EA 2D E2 F0 CE C8 35 C9 24 44 12 B0 B9 58
Start Right key:
 F6 87 AA 3C 2F 34 A9 F0 2F 1A 9D 5F BB E7 4C 6B
C_i:
 C9 E8 81 9D C7 3B A5 AE 50 F5 B5 70 56 1A 6A 07
After xor C_i:
 64 F6 6B B0 25 CB 6B 66 65 3C 91 34 44 AA D3 5F
After apply S_Box:
 10 B4 6F AD 5C E4 6F 9A 7B 7F 0F E3 EA 38 49 87
After apply L:
 3B 78 7A 12 4D 16 30 A9 59 C2 41 B4 AC 90 BD 47
Left key inter 7:
 CD FF D0 2E 62 22 99 59 76 D8 DC EB 17 77 F1 2C
Right key inter 7:
 AD 1E EA 2D E2 F0 CE C8 35 C9 24 44 12 B0 B9 58


In [547]:
def expand_key(key: bitarray) -> list:
    left, right = key[:128], key[128:]
    for i in range(1, 9):
        C_i = generate_C(i)
        left, right = RoundKeys(left, right, C_i)
    return left, right

# Раундовое преобразование

In [548]:
def round_Kuznechik(state: bitarray, key: bitarray) -> bitarray:
    print("state:\n", bin_to_hex(state))
    state = apply_X(state, key)
    print("After xor key:\n", bin_to_hex(state))
    state = apply_S_box(state)
    print("After apply S_Box:\n", bin_to_hex(state))
    state = apply_L(state)
    print("After apply L:\n", bin_to_hex(state))
    return state

In [551]:
state = '3D 20 B6 5F 3E 17 86 AA E9 B8 9F 37 E7 CA 81 3D'
round_key = '79 F1 C9 F2 FD 49 05 47 B8 F4 32 05 97 A6 F2 1B'
state_bin = hex_to_bin(state, version=128)
round_key = hex_to_bin(round_key, version=128)
res = round_Kuznechik(state_bin, round_key)

state:
 3D 20 B6 5F 3E 17 86 AA E9 B8 9F 37 E7 CA 81 3D
After xor key:
 44 D1 7F AD C3 5E 83 ED 51 4C AD 32 70 6C 73 26
After apply S_Box:
 EA 1B 57 9F 40 5D A9 E5 70 FD 9F 02 32 9D 3D EF
After apply L:
 D4 8E DE 32 C8 E6 7E 10 49 89 E1 49 29 D3 F9 46
